In [1]:
# Cell 1: Imports and Environment Check
import os
import sys
import subprocess
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

# Check if we are running in the right environment
print(f"✅ Python Location: {sys.executable}")

# Try importing the tricky libraries
try:
    from meeko import MoleculePreparation, PDBQTWriterLegacy
    print("✅ Meeko Loaded.")
except ImportError:
    print("❌ Meeko is missing! Did you activate 'docking_env'?")

# Check if Vina is accessible
try:
    vina_version = subprocess.check_output(["vina", "--version"], text=True).strip()
    print(f"✅ Vina Found: {vina_version}")
except:
    print("❌ Vina not found in path. Make sure you installed it in Conda.")

✅ Python Location: /home/skd/miniconda3/envs/docking_env/bin/python
✅ Meeko Loaded.
✅ Vina Found: AutoDock Vina 7ac2999-mod


In [2]:
# Cell 2: Configuration
# The ID of the protein we are targeting (Carbonic Anhydrase)
PDB_ID = "3HS4"

# Define the output directory relative to this notebook
# We go one level up (..) and into 'docking_results'
OUTPUT_DIR = os.path.join("..", "docking_results")

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"📂 Created directory: {OUTPUT_DIR}")
else:
    print(f"📂 Using directory: {OUTPUT_DIR}")

📂 Created directory: ../docking_results


In [4]:
# Cell 3: Receptor Preparation (Fixed & Cleaned)
import os
import subprocess

protein_pdb = os.path.join(OUTPUT_DIR, f"{PDB_ID}.pdb")
protein_clean_pdb = os.path.join(OUTPUT_DIR, f"{PDB_ID}_clean.pdb")
protein_pdbqt = os.path.join(OUTPUT_DIR, f"{PDB_ID}.pdbqt")

# 1. Download PDB
if not os.path.exists(protein_pdb):
    print(f"⬇️ Downloading {PDB_ID}...")
    subprocess.run(f"wget -q https://files.rcsb.org/download/{PDB_ID}.pdb -O {protein_pdb}", shell=True)

# 2. CLEAN THE PDB (Fixes the '0 molecules converted' error)
# We strictly keep only protein atoms (ATOM) and ignore waters/ions that confuse OpenBabel
print("🧹 Cleaning PDB file...")
with open(protein_pdb, "r") as f_in, open(protein_clean_pdb, "w") as f_out:
    for line in f_in:
        if line.startswith("ATOM") or line.startswith("TER"):
            f_out.write(line)
print(f"✅ Cleaned PDB saved to: {protein_clean_pdb}")

# 3. Convert CLEAN PDB to PDBQT
# We overwrite the old bad file if it exists
if os.path.exists(protein_pdbqt):
    os.remove(protein_pdbqt)

print("⚙️ Converting Protein to PDBQT...")
# We use the clean file now
cmd = [
    "obabel", protein_clean_pdb, "-O", protein_pdbqt, 
    "-xr", "-p", "7.4", "--partialcharge", "gasteiger"
]
result = subprocess.run(cmd, capture_output=True, text=True)

# Verify it worked
if os.path.exists(protein_pdbqt) and os.path.getsize(protein_pdbqt) > 0:
    print(f"✅ Success! Receptor ready: {protein_pdbqt}")
else:
    print("❌ Conversion Failed. OpenBabel Output:")
    print(result.stderr)

🧹 Cleaning PDB file...
✅ Cleaned PDB saved to: ../docking_results/3HS4_clean.pdb
⚙️ Converting Protein to PDBQT...
✅ Success! Receptor ready: ../docking_results/3HS4.pdbqt


In [5]:
# Cell 4: Define Ligands
candidates = [
    # The absolute best candidate (pIC50 ~8.41)
    {"name": "Rank_10", "smiles": "Cc1ccc(NS(=O)(=O)Nc2ccccc2)c(c=O)n1CC(=O)NCc1ccccc1"},
    
    # Other high performers
    {"name": "Rank_4",  "smiles": "CC(C)(CN(CC(=O)NC(CCNN=C(N)N)B(O)O)C(=O)CCc1ccccc1)C(=O)O"},
    {"name": "Rank_7",  "smiles": "CC1(C)C2CC3OB(C(CCCCN)NC(=O)C4CCCN4C(=O)c4cccc(C#N)c4)OC3C21"},
    {"name": "Rank_2",  "smiles": "CC1(F)C2CC3OB(C(CCCCN)NC(=O)C4CCCN4C(=O)c4cccc(C#N)c4)OC3C21"}
]

print(f"📋 Loaded {len(candidates)} candidates for validation.")

📋 Loaded 4 candidates for validation.


In [13]:
# Cell 5: Docking Simulation (With Boron Compatibility Fix)
import os
import subprocess
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from meeko import MoleculePreparation, PDBQTWriterLegacy

# --- 1. RE-CHECK RECEPTOR ---
protein_pdbqt = os.path.join(OUTPUT_DIR, f"{PDB_ID}.pdbqt")
if not os.path.exists(protein_pdbqt):
    print("❌ Receptor PDBQT missing. Run Cell 3.")
else:
    print(f"✅ Receptor Ready: {protein_pdbqt}")

# --- 2. RUN DOCKING LOOP ---
results = []
print(f"\n🚀 STARTING DOCKING ({len(candidates)} Candidates)...")

for cand in candidates:
    name = cand['name']
    smiles = cand['smiles']
    print(f"👉 Docking {name}...", end=" ")
    
    try:
        # --- THE BORON FIX ---
        # Vina crashes on Boron (B). We replace it with Carbon (C) 
        # to approximate the geometry and get a score.
        if "B" in smiles:
            print("(Found Boron -> Simulating as Carbon)", end=" ")
            smiles = smiles.replace("B", "C")
            
        # A. Prepare Ligand (RDKit)
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            print("❌ Invalid SMILES")
            continue
            
        # Sanitization
        try:
            Chem.SanitizeMol(mol)
            mol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol, AllChem.ETKDG())
        except:
            print("❌ Chemistry Error (Skipping)")
            continue

        # B. Prepare PDBQT (Meeko v0.5)
        prep = MoleculePreparation()
        mol_setups = prep.prepare(mol)
        setup = mol_setups[0] 
        
        # Write PDBQT
        write_output = PDBQTWriterLegacy.write_string(setup)
        if isinstance(write_output, tuple):
            ligand_string = write_output[0]
        else:
            ligand_string = write_output
        
        ligand_file = os.path.join(OUTPUT_DIR, f"{name}.pdbqt")
        out_file = os.path.join(OUTPUT_DIR, f"{name}_out.pdbqt")
        
        with open(ligand_file, "w") as f:
            f.write(ligand_string)
            
        # C. Run Vina
        cmd = [
            "vina",
            "--receptor", protein_pdbqt,
            "--ligand", ligand_file,
            "--center_x", "14.5", "--center_y", "-7.5", "--center_z", "3.5",
            "--size_x", "22", "--size_y", "22", "--size_z", "22",
            "--out", out_file,
            "--exhaustiveness", "8"
        ]
        
        proc = subprocess.run(cmd, capture_output=True, text=True)
        
        # D. Get Score
        score = None
        for line in proc.stdout.splitlines():
            if line.strip().startswith("1"):
                score = float(line.split()[1])
                break
        
        if score:
            print(f"🎉 Score: {score}")
            results.append({"Name": name, "Affinity": score})
        else:
            print("⚠️ Failed.")
            # print(proc.stderr) # Uncomment if you need to debug again

    except Exception as e:
        print(f"❌ Crash: {e}")

print("\n🏆 RESULTS:")
if results:
    display(pd.DataFrame(results).sort_values("Affinity"))
else:
    print("No valid results.")

[RDKit] ERROR:[12:45:53] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 16


✅ Receptor Ready: ../docking_results/3HS4.pdbqt

🚀 STARTING DOCKING (4 Candidates)...
👉 Docking Rank_10... ❌ Invalid SMILES
👉 Docking Rank_4... (Found Boron -> Simulating as Carbon) 🎉 Score: -6.486
👉 Docking Rank_7... (Found Boron -> Simulating as Carbon) 🎉 Score: -7.971
👉 Docking Rank_2... (Found Boron -> Simulating as Carbon) 🎉 Score: -7.896

🏆 RESULTS:


,Name,Affinity
1,Rank_7,-7.971
2,Rank_2,-7.896
0,Rank_4,-6.486


In [14]:
# Cell 7: Install 3D Visualization Tool
import sys
import subprocess

print("⬇️ Installing py3Dmol...")
subprocess.run([sys.executable, "-m", "pip", "install", "py3Dmol"], check=True)
print("✅ Ready to visualize!")

⬇️ Installing py3Dmol...
✅ Ready to visualize!


In [15]:
# Cell 8: Visualize the Best Docking Result
import py3Dmol
import os

# Select your best candidate
best_drug = "Rank_7" 

# Paths to files
protein_path = os.path.join(OUTPUT_DIR, f"{PDB_ID}.pdb") # Use PDB, not PDBQT for visualizer
ligand_path = os.path.join(OUTPUT_DIR, f"{best_drug}_out.pdbqt")

print(f"👀 Visualizing {best_drug} binding to {PDB_ID}...")

# 1. Initialize Viewer
view = py3Dmol.view(width=800, height=600)

# 2. Load Protein (Gray Cartoon)
with open(protein_path, 'r') as f:
    view.addModel(f.read(), "pdb")
view.setStyle({'model': -1}, {"cartoon": {'color': 'gray'}})

# 3. Load Drug (Green Sticks)
with open(ligand_path, 'r') as f:
    view.addModel(f.read(), "pdbqt")
view.setStyle({'model': -1}, {"stick": {'colorscheme': 'greenCarbon'}})

# 4. Zoom and Show
view.zoomTo()
view.show()

👀 Visualizing Rank_7 binding to 3HS4...


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [16]:
# Cell 9: High-Quality Visualization (Fix for Invisible Drug)
import py3Dmol
import os
import subprocess

# 1. Select your best result
best_drug = "Rank_7"
output_pdbqt = os.path.join(OUTPUT_DIR, f"{best_drug}_out.pdbqt")
output_pdb = os.path.join(OUTPUT_DIR, f"{best_drug}_docked.pdb")
protein_pdb = os.path.join(OUTPUT_DIR, f"{PDB_ID}.pdb")

# 2. Convert the Docked Result to PDB format (Easier to see)
print(f"🔄 Converting {best_drug} result to PDB for viewing...")
subprocess.run(f"obabel {output_pdbqt} -O {output_pdb}", shell=True)

# 3. Setup Viewer
print(f"👀 Generating 3D Interaction Model...")
view = py3Dmol.view(width=800, height=600)

# A. Load Protein (Grey Cartoon - Surface optional)
with open(protein_pdb, 'r') as f:
    view.addModel(f.read(), "pdb")
view.setStyle({'model': 0}, {"cartoon": {'color': '#eeeeee'}}) # Light Grey Protein

# B. Load Drug (Green Sticks)
if os.path.exists(output_pdb):
    with open(output_pdb, 'r') as f:
        view.addModel(f.read(), "pdb")
    # Style the drug (Model 1) to be Green and thick
    view.setStyle({'model': 1}, {"stick": {'colorscheme': 'greenCarbon', 'radius': 0.3}})
else:
    print("❌ Error: Could not convert ligand file.")

# 4. Zoom and Render
view.zoomTo()
view.show()

🔄 Converting Rank_7 result to PDB for viewing...
👀 Generating 3D Interaction Model...


9 molecules converted


3Dmol.js failed to load for some reason. Please check your browser console for error messages.